In [2]:
from bs4 import BeautifulSoup
import requests
import pandas as pd

# 1st link

In [2]:
headers = {
    'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10.12; rv:55.0) Gecko/20100101 Firefox/55.0',
}

content = requests.get('https://www.nrcs.usda.gov/wps/portal/nrcs/main/soils/survey/class/maps/', headers=headers).text
soilSoup = BeautifulSoup(content, 'html.parser')

In [3]:
soilList = soilSoup.find("div", class_="centerColImg").find_all("h3")
soil_types = []
dfsoil1 = pd.DataFrame()
for item in soilList[1:]:
    soil_types.append(item.get_text().replace('\n', '').replace('\r', '').replace('\t', ''))
dfsoil1 = pd.DataFrame({"Soil":soil_types, "Parent": "Soil", "Relation": "subClass"})
print(dfsoil1)

           Soil Parent  Relation
0      Alfisols   Soil  subClass
1      Andisols   Soil  subClass
2     Aridisols   Soil  subClass
3      Entisols   Soil  subClass
4      Gelisols   Soil  subClass
5     Histosols   Soil  subClass
6   Inceptisols   Soil  subClass
7     Mollisols   Soil  subClass
8       Oxisols   Soil  subClass
9     Spodosols   Soil  subClass
10     Ultisols   Soil  subClass
11    Vertisols   Soil  subClass


# 2nd link

In [5]:
headers = {
    'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10.12; rv:55.0) Gecko/20100101 Firefox/55.0',
}

content = requests.get('https://sites.google.com/site/dinpuithai/Home/key10', headers=headers).text
soilSoup2 = BeautifulSoup(content, 'html.parser')

In [6]:
def traverse1(ul):
    result = {}
    for li in ul.find_all("li", recursive=False):
        key = next(li.stripped_strings)
        HasUL = li.find("ul")
        #If you want only the node text
        #result[key] = None if HasUL is None else traverse1(HasUL)
        result[key] = key
    return result

In [7]:
dictionary = {}
soil = []
parent = []
for item in soilList2:
    soilCategory = item.find("div").text.split(".")[1]
    soil.append(soilCategory)
    parent.append("Soil")
    dictionary = traverse1(item.find("ul"))
    for key, value in dictionary.items():
        soil.append(key.split(".")[1])
        parent.append(soilCategory)
        if type(value) is dict:
            for key1, value1 in value.items():
                soil.append(key1.split(".")[1])
                parent.append(key.split(".")[1])
    dfnewsoil = pd.DataFrame({"Soil":soil, "Parent": parent, "Relation": "subClass"})
print(dfnewsoil)

         Soil    Parent  Relation
0    Gelisols      Soil  subClass
1     Histels  Gelisols  subClass
2     Turbels  Gelisols  subClass
3     Orthels  Gelisols  subClass
4   Histosols      Soil  subClass
..        ...       ...       ...
75    Aquents  Entisols  subClass
76     Arents  Entisols  subClass
77  Psamments  Entisols  subClass
78   Fluvents  Entisols  subClass
79   Orthents  Entisols  subClass

[80 rows x 3 columns]


# Data Aggregation 

In [8]:
frames = [dfsoil1, dfnewsoil]

In [9]:
result = pd.concat(frames)

In [10]:
print(result)

         Soil    Parent  Relation
0    Alfisols      Soil  subClass
1    Andisols      Soil  subClass
2   Aridisols      Soil  subClass
3    Entisols      Soil  subClass
4    Gelisols      Soil  subClass
..        ...       ...       ...
75    Aquents  Entisols  subClass
76     Arents  Entisols  subClass
77  Psamments  Entisols  subClass
78   Fluvents  Entisols  subClass
79   Orthents  Entisols  subClass

[92 rows x 3 columns]


In [11]:
result1 = result.drop_duplicates(['Soil'])

In [12]:
result1.to_excel("outputsoilremovingDuplicatesNEW.xlsx") 
result1.to_csv("outputsoilremovingDuplicatesNEW.csv", index=False) 

# Save to existing ontology

In [1]:
import pandas as pd
from owlready2 import *
onto = get_ontology("file://C:/Users/Lenovo/Desktop/UNI-Koblenz-Landua/Research/Coding/geological-taxonomy.owl").load()
with onto:
    class Soil(onto.Geological_Object):
        pass

In [ ]:
data = pd.read_csv("outputsoilremovingDuplicates.csv")

In [ ]:
for index, row in data.iterrows():
    with onto:
        soil = row['Soil']
        parent = row['Parent']
        for soilCategory in list(onto.classes()):
            if (parent == soilCategory.name):
                NewClass = type(soil,(soilCategory,),{})

In [ ]:
for index, row in data.iterrows():
    with onto:
        soil = row['Soil']
        parent = row['Parent']
        for k in list(onto.classes()):
            if(k.name==parent):
                parent_class = k
                break      
        NewClass = type(soil,(k,),{})

In [ ]:
onto.save(file = "./geological-taxonomy.owl", format = "rdfxml")